In [58]:
import tensorflow as tf
import tensorflow.keras
from unet import unet_model_3d
import os
from data import get_train_and_validation_datasets

In [59]:
PROJECT = 'freelearing'
BUCKET = 'postgraduate_ai'
REGION = 'europe-west1'
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [46]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [49]:
%%bash
# This command will fail if the Cloud Machine Learning Engine API is not enabled using the link above.
echo "Getting the service account email associated with the Cloud Machine Learning Engine API"

AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print (response['serviceAccount'])")  # If this command fails, the Cloud Machine Learning Engine API has not been enabled above.

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET   
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET   # error message (if bucket is empty) can be ignored.  
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

Getting the service account email associated with the Cloud Machine Learning Engine API
Authorizing the Cloud ML Service account service-587080046851@cloud-ml.google.com.iam.gserviceaccount.com to access files in postgraduate_ai


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   235    0   235    0     0    286      0 --:--:-- --:--:-- --:--:--   286
Updated default ACL on gs://postgraduate_ai/
Updated ACL on gs://postgraduate_ai/._Task07_Pancreas
Updated ACL on gs://postgraduate_ai/test_iris.csv
Updated ACL on gs://postgraduate_ai/34/1a3b5244f213282b7b0920b729c592
Updated ACL on gs://postgraduate_ai/Task07_Pancreas/imagesTr/._pancreas_004.nii.gz
Updated ACL on gs://postgraduate_ai/Task07_Pancreas/imagesTr/pancreas_016.nii.gz
Updated ACL on gs://postgraduate_ai/Task07_Pancreas/imagesTr/pancreas_001.nii.gz
Updated ACL on gs://postgraduate_ai/59/8f609e25bd075d5330ba0116d17806.dir
Updated ACL on gs://postgraduate_ai/Task07_Pancreas/.DS_Store
Updated ACL on gs://postgraduate_ai/Task07_Pancreas/imagesTr/._pancreas_001.nii.gz
Updated ACL on gs://postgraduate_ai/Task07_Pancreas/imagesTr/._.DS_Store
Updated

In [60]:
input_shape = (1,40,40,40)
model = unet_model_3d(input_shape = input_shape, n_labels = 1, activation_name = 'relu')
#model.summary()

In [68]:
batch_size = 1
#path_images = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data", "resized_dataset", "imagesTr")
path_images = tf.io.gfile.glob("gs://postgraduate_ai/resized_dataset/imagesTr/*.gz")
print(path_images)

#path_labels = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data", "resized_dataset", "labelsTr")
path_labels = tf.io.gfile.glob("gs://postgraduate_ai/resized_dataset/labelsTr/*.gz")
print(path_labels)

['gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_001.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_004.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_005.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_006.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_010.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_012.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_015.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_016.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_018.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_019.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_021.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_024.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_025.nii.gz', 'gs://postgraduate_ai/resized_dataset/imagesTr/pancreas_028.nii.gz', 'gs://postgraduate_ai/resized_dat

In [69]:
train_dataset, validation_dataset, validation_images = get_train_and_validation_datasets(
    0.2,
    path_images,
    path_labels,
    patch=True,
    patch_shape=(40,40,40)
)

train_dataset = train_dataset.shuffle(100).batch(batch_size)
validation_dataset = validation_dataset.batch(batch_size)

TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not list

In [21]:
epochs = 1
#model.fit(train_dataset, epochs = epochs, validation_data = validation_dataset)
model.fit(train_dataset, epochs = epochs)

------- Image to be processed:
/home/jupyter/ai_postgraduate_project/data/resized_dataset/imagesTr/pancreas_001.nii.gz
------- Image to be processed:
/home/jupyter/ai_postgraduate_project/data/resized_dataset/labelsTr/pancreas_001.nii.gz
    155/Unknown - 34s 220ms/step - loss: -0.9806 - dice_coefficient: 0.9806

UnknownError: 2 root error(s) found.
  (0) Unknown:  IndexError: index 507 is out of bounds for axis 0 with size 507
Traceback (most recent call last):

  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/script_ops.py", line 236, in __call__
    ret = func(*args)

  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/data/ops/dataset_ops.py", line 789, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/jupyter/ai_postgraduate_project/notebooks/cpachon/data.py", line 150, in data_iterator
    label, indices, index, finished = next_patch(big_label,patch_shape,indices,index)

  File "/home/jupyter/ai_postgraduate_project/notebooks/cpachon/data.py", line 112, in next_patch
    img = get_patch_from_3d_data(img,patch_shape,indices[index])

IndexError: index 507 is out of bounds for axis 0 with size 507


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_4]]
  (1) Unknown:  IndexError: index 50